## Imports

In [1]:
import pandas as pd
import itertools
from gurobipy import *

In [2]:
def readData():
    xls = pd.ExcelFile("data.xlsx")
    
    # Getting point coordinates
    sheet = xls.parse(0) 
    
    xCoordinates = list(sheet['X-coordinate'])
    yCoordinates = list(sheet['Y-coordinate'])
    points = [(xCoordinates[i], yCoordinates[i]) for i in range(len(xCoordinates))]
    
    # Getting block data
    sheet = xls.parse(1, skiprows = [0])
    
    xCoordinates = list(sheet['X coordinate'])
    yCoordinates = list(sheet['Y coordinate'])
    widths = list(sheet['Width'])
    heights = list(sheet['Height'])
    blocks = [(xCoordinates[i], yCoordinates[i], widths[i], heights[i]) for i in range(len(xCoordinates))]
    
    return points, blocks

In [3]:
def manhattanDistance(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    
    return abs(x1 - x2) + abs(y1 - y2)

In [4]:
def calculate_C_Distances(points):
    Cij = []
    for i in range(0, len(points)):
        Cj = []
        for j in range(0, len(points)):
            Cj.append(manhattanDistance(points[i], points[j]))
        Cij.append(Cj)

    return Cij

## Parameters

In [5]:
points, blocks = readData()
N = len(points)
C = calculate_C_Distances(points)

## Sub-tour elimination

In [6]:
def subtourelim(model, where):
    if where == GRB.callback.MIPSOL:
        selected = []
        # make a list of edges selected in the solution
        for i in range(N):
            sol = model.cbGetSolution([model._vars[i,j] for j in range(N)])
            selected += [(i,j) for j in range(N) if sol[j] > 0.5]
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < N:
            # add a subtour elimination constraint
            expr = 0
            for i in range(len(tour)):
                for j in range(i+1, len(tour)):
                    expr += model._vars[tour[i], tour[j]]
            model.cbLazy(expr <= len(tour)-1)
    
def subtour(edges):
    visited = [False]*N
    cycles = []
    lengths = []
    selected = [[] for i in range(N)]
    for x,y in edges:
        selected[x].append(y)
    while True:
        current = visited.index(False)
        thiscycle = [current]
        while True:
            visited[current] = True
            neighbors = [x for x in selected[current] if not visited[x]]
            if len(neighbors) == 0:
                break
            current = neighbors[0]
            thiscycle.append(current)
        cycles.append(thiscycle)
        lengths.append(len(thiscycle))
        if sum(lengths) == N:
            break
    return cycles[lengths.index(min(lengths))]

## Creating model

In [7]:
model = Model()

Using license file /Users/fuadaghazada/gurobi.lic
Academic license - for non-commercial use only


## Decision Variables

In [8]:
decVars = {}
for i in range(N):
    for j in range(N):
        decVars[i, j] = model.addVar(vtype = GRB.BINARY, name = f"X_{i}_{j}")
    model.update()

## Constraints

In [9]:
for i in range(N):
    decVars_j = []
    for j in range(N):
        decVars_j.append(j)
    
    model.addConstr(sum(decVars[i, k] for k in decVars_j), GRB.EQUAL, 1)
model.update()

In [10]:
for j in range(N):
    decVars_i = []
    for i in range(N):
        decVars_i.append(i)
    
    model.addConstr(sum(decVars[k, j] for k in decVars_i), GRB.EQUAL, 1)
model.update()

In [11]:
for i in range(N):
    model.addConstr(decVars[i, i], GRB.EQUAL, 0)
model.update()

## Objective function

In [12]:
obj = 0
for i in range(N):
    for j in range(N):
        obj += C[i][j] * decVars[i, j] 

In [13]:
model.update()
model._vars = decVars
model.setObjective(obj, GRB.MINIMIZE)
model.params.LazyConstraints = 1
model.optimize(subtourelim)

Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 150 rows, 2500 columns and 5050 nonzeros
Model fingerprint: 0x35de07cc
Variable types: 0 continuous, 2500 integer (2500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 50 rows and 50 columns
Presolve time: 0.01s
Presolved: 100 rows, 2450 columns, 4900 nonzeros
Variable types: 0 continuous, 2450 integer (2450 binary)
Found heuristic solution: objective 316.0000000

Root relaxation: objective 2.960000e+02, 92 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     296.0000000  296.00000  0.00%     -    0s

Explored 0 nodes (92 simplex iterat

In [14]:
for v in model.getVars():
    if v.X != 0:
        print("%s %f" % (v.Varname, v.X))

X_0_13 1.000000
X_1_16 1.000000
X_2_32 1.000000
X_3_18 1.000000
X_4_27 1.000000
X_5_36 1.000000
X_6_40 1.000000
X_7_22 1.000000
X_8_7 1.000000
X_9_5 1.000000
X_10_11 1.000000
X_11_10 1.000000
X_12_39 1.000000
X_13_0 1.000000
X_14_28 1.000000
X_15_26 1.000000
X_16_1 1.000000
X_17_25 1.000000
X_18_37 1.000000
X_19_46 1.000000
X_20_30 1.000000
X_21_34 1.000000
X_22_8 1.000000
X_23_24 1.000000
X_24_35 1.000000
X_25_17 1.000000
X_26_48 1.000000
X_27_47 1.000000
X_28_14 1.000000
X_29_41 1.000000
X_30_20 1.000000
X_31_45 1.000000
X_32_2 1.000000
X_33_38 1.000000
X_34_21 1.000000
X_35_23 1.000000
X_36_15 1.000000
X_37_3 1.000000
X_38_33 1.000000
X_39_12 1.000000
X_40_6 1.000000
X_41_42 1.000000
X_42_29 1.000000
X_43_9 1.000000
X_44_49 1.000000
X_45_31 1.000000
X_46_19 1.000000
X_47_4 1.000000
X_48_43 1.000000
X_49_44 1.000000


In [15]:
model.status == GRB.Status.OPTIMAL

True

In [16]:
from itertools import chain, combinations
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))